In [ ]:
import sys
import collections
import dpkt
import socket
import os
import random
import math
import numpy as np
from scipy.stats import kurtosis, skew
from termcolor import colored
from joblib import Parallel, delayed

# Extract UDP DATA

# Extract TCP DATA

In [ ]:
import processing_TCP
# data_folder = "size_defence_action_data/800padded"
# data_folder = "action_data"
data_folder = "sawyer_action_data/Position"
dst_folder = "extractedFeatures/Position"
# DEST_IP = '192.168.1.1'
DEST_IP = '192.168.0.100'
# SOURCE_IP = '192.168.1.10'
SOURCE_IP = '192.168.0.103'
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]
for action in all_actions:
    processing_TCP.FeatureExtractionCombined(data_folder, DEST_IP, SOURCE_IP, dst_folder, action)
    

# EXTRACT TEMPORAL DEPENDENCIES

In [ ]:
import os
import numpy as np
import pandas as pd
import temp_dep_extractor

# data_folder = 'size_defence_action_data/800padded'
data_folder = 'action_data'

# data_folder = "action_data"
# Get a list of all directories in the data_folder
all_actions = [d for d in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, d))]

## EXTRACT CARTESIAN COMMANDS STATISTICS

In [ ]:
kernel = np.array([154, 52, 90, 81, 52, 81, 725, 52, 81, 90, 157, 81, 154])

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'cart')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)
        statistics = temp_dep_extractor.extract_conv_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)
        

## EXTRACT GRIPPER SPEED COMMANDS STATISTICS

In [ ]:
kernel = np.array([104, 111, 104, 111, 104, 111])

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'gripper_speed')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)

        # Extract features
        statistics = temp_dep_extractor.extract_corr_coeff_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)


## EXTRACT GRIPPER POSITION COMMANDS STATISTICS

In [ ]:
kernel = np.array([168,66,104,95,66,104,95,66,104,390,95,104,111,95,66,95,95,66,170])
kernel = [x - 14 for x in kernel]

for action_name in all_actions:
    extraction_dir = os.path.join('extractedFeatures', action_name, 'gripper_position')
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)

    dir_path = os.path.join(data_folder, action_name)
    files = os.listdir(dir_path)
    
    for file in files:
        if (".DS_Store" in file):
            continue
        print(file)
        pcap_file_path = os.path.join(dir_path, file)
        timestamps, packet_sizes = temp_dep_extractor.parse_pcap(pcap_file_path)
        statistics = temp_dep_extractor.extract_conv_features(timestamps, packet_sizes, file[:-5], action_name, kernel)
        df = pd.DataFrame(statistics, index=[0])
        
        output_file_path = os.path.join(extraction_dir, f'{os.path.splitext(file)[0]}.csv')
        df.to_csv(output_file_path, index=False)
        

In [ ]:
res = temp_dep_extractor.custom_convolve([3,1,2,4,1,3,2,10,12,3,4,4],[2,4,1])
print(res)

# TRAIN CLASSIFIER

In [ ]:
!python traffic_classifier.py "/path/to/directory_path" "['class1', 'class2']"